# Loading Dataset

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("../Datasets/Cars/cars.csv")
df.sample(5)

,brand,km_driven,fuel,owner,selling_price
4967,Tata,15000,Petrol,First Owner,790000
3257,Maruti,36659,Petrol,First Owner,160000
6842,Mahindra,110000,Diesel,First Owner,650000
7207,Hyundai,60000,Petrol,Second Owner,280000
5245,Audi,31800,Diesel,First Owner,3975000


In [3]:
df["brand"].value_counts()

Maruti           2448
Hyundai          1415
Mahindra          772
Tata              734
Toyota            488
Honda             467
Ford              397
Chevrolet         230
Renault           228
Volkswagen        186
BMW               120
Skoda             105
Nissan             81
Jaguar             71
Volvo              67
Datsun             65
Mercedes-Benz      54
Fiat               47
Audi               40
Lexus              34
Jeep               31
Mitsubishi         14
Force               6
Land                6
Isuzu               5
Kia                 4
Ambassador          4
Daewoo              3
MG                  3
Ashok               1
Opel                1
Peugeot             1
Name: brand, dtype: int64

# OneHotEncoding Using Pandas

In [4]:
pandas_df = pd.get_dummies(df, columns = ["fuel", "owner"])
pandas_df.head()

,brand,km_driven,selling_price,fuel_CNG,fuel_Diesel,fuel_LPG,fuel_Petrol,owner_First Owner,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,Maruti,145500,450000,0,1,0,0,1,0,0,0,0
1,Skoda,120000,370000,0,1,0,0,0,0,1,0,0
2,Honda,140000,158000,0,0,0,1,0,0,0,0,1
3,Hyundai,127000,225000,0,1,0,0,1,0,0,0,0
4,Maruti,120000,130000,0,0,0,1,1,0,0,0,0


In [5]:
print(df.shape)
print(pandas_df.shape)

(8128, 5)
(8128, 12)


# K-1 OneHotEncoding

In [6]:
pandas_df = pd.get_dummies(df, columns = ["fuel", "owner"], drop_first = True)
pandas_df.head()

,brand,km_driven,selling_price,fuel_Diesel,fuel_LPG,fuel_Petrol,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,Maruti,145500,450000,1,0,0,0,0,0,0
1,Skoda,120000,370000,1,0,0,0,1,0,0
2,Honda,140000,158000,0,0,1,0,0,0,1
3,Hyundai,127000,225000,1,0,0,0,0,0,0
4,Maruti,120000,130000,0,0,1,0,0,0,0


In [7]:
print(pandas_df.shape)

(8128, 10)


# OneHotEncoding Using Scikit Learn

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.iloc[:, 0:4],
    df.iloc[:, -1],
    test_size = 0.2,
    random_state = 0
)

In [12]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(drop = "first", sparse = False, dtype = np.int32)  # drop="first" to drop one encoded column

X_train_new = ohe.fit_transform(X_train[["fuel", "owner"]])
x_test_new  = ohe.transform(X_test[["fuel", "owner"]])

print(X_train_new.shape)

(6502, 7)


In [13]:
# now have to merge the rest 2 columns with the new OneHotEncoded array
np.hstack((X_train[["brand", "km_driven"]].values, X_train_new))

array([['Hyundai', 60000, 0, ..., 0, 0, 0],
       ['Tata', 150000, 1, ..., 0, 0, 1],
       ['Hyundai', 110000, 1, ..., 1, 0, 0],
       ...,
       ['Hyundai', 90000, 0, ..., 1, 0, 0],
       ['Volkswagen', 90000, 1, ..., 0, 0, 0],
       ['Hyundai', 110000, 0, ..., 0, 0, 0]], dtype=object)

# OneHotEncoding with Top Categories

In [14]:
counts = df["brand"].value_counts()

In [15]:
df["brand"].nunique()
threshold = 100

In [17]:
repl = counts[counts <= threshold].index

In [18]:
pd.get_dummies(df["brand"].replace(repl, "uncommon"))

,BMW,Chevrolet,Ford,Honda,Hyundai,Mahindra,Maruti,Renault,Skoda,Tata,Toyota,Volkswagen,uncommon
0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8123,0,0,0,0,1,0,0,0,0,0,0,0,0
8124,0,0,0,0,1,0,0,0,0,0,0,0,0
8125,0,0,0,0,0,0,1,0,0,0,0,0,0
8126,0,0,0,0,0,0,0,0,0,1,0,0,0
